# Indexing: Text Embedding with OpenAI

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
pip show langchain

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Users\Marcus\anaconda3\envs\langchain_env_py312\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv

In [3]:
# Import necessary modules and classes
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
import numpy as np

In [4]:
# Create instance of loader class and upload the text document
loader_docx = Docx2txtLoader("Introduction_to_Data_and_Data_Science_2.docx")
pages = loader_docx.load()

# Initialize markdown header text splitter
md_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on = [("#", "Course Title"), 
                           ("##", "Lecture Title")]
)

# Split text into lectures
pages_md_split = md_splitter.split_text(pages[0].page_content)

# Implement for loop to remove newline characters
for i in range(len(pages_md_split)):
    pages_md_split[i].page_content = ' '.join(pages_md_split[i].page_content.split())
    
# Create text splitter object to split text into chuncks
char_splitter = CharacterTextSplitter(
    separator = ".",
    chunk_size = 500,
    chunk_overlap  = 50
)

# Split the document with character splitter
pages_char_split = char_splitter.split_documents(pages_md_split)

In [6]:
# Display the pages_char_split variable
pages_char_split

[Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis'),
 Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to digest chunks and study them individu

In [7]:
# Intialize embedding object with OpenAIEmbeddings class and model
embedding = OpenAIEmbeddings(model = "text-embedding-ada-002")

In [8]:
# Review content to embed - not 3,5 from same lecture and should have close vector relationship
pages_char_split[3]

Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='Analytics is essentially the application of logical and computational reasoning to the component parts obtained in an analysis. And in doing this you are looking for patterns and exploring what you could do with them in the future. Here, analytics branches off into two areas: qualitative analytics – this is using your intuition and experience in conjunction with the analysis to plan your next business move')

In [9]:
pages_char_split[5]

Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='You may use this intuition to decide on which styles of clothing to start selling. This would be qualitative analytics. But you might not know when to introduce the new collection. In that case, relying on past sales data and user experience data, you could predict in which month it would be best to do that. This is an example of using quantitative analytics')

In [10]:
pages_char_split[18]

Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}, page_content='More importantly, it will be sufficient for your need to create quick and accurate analyses. However, if your theoretical preparation is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on')

In [12]:
# Create vectors with embedding object
vector1 = embedding.embed_query(pages_char_split[3].page_content)
vector2 = embedding.embed_query(pages_char_split[5].page_content)
vector3 = embedding.embed_query(pages_char_split[18].page_content)

In [13]:
# Review vector
vector1

[0.003742626868188381,
 0.010484526865184307,
 0.010070833377540112,
 -0.03043227642774582,
 -0.01968919113278389,
 0.012074658647179604,
 -0.024239812046289444,
 -0.013018394820392132,
 0.010562093928456306,
 -0.027096876874566078,
 0.004783323034644127,
 0.01674162968993187,
 -0.013115353882312775,
 0.006341134663671255,
 -0.010898219421505928,
 -0.01657356694340706,
 0.03604298457503319,
 -0.0008976810495369136,
 0.020270947366952896,
 -0.02367098443210125,
 -0.04374801367521286,
 0.022494545206427574,
 -0.00870047602802515,
 -0.027407146990299225,
 -0.009999730624258518,
 0.0037006111815571785,
 0.010258288122713566,
 -0.0250284131616354,
 0.004815642721951008,
 -0.01821541041135788,
 0.009734708815813065,
 0.005672115832567215,
 -0.008002369664609432,
 0.0004969161236658692,
 -0.009055993519723415,
 0.0029475612100213766,
 0.011143849231302738,
 0.01587546057999134,
 0.015293705277144909,
 0.017633654177188873,
 0.01521613821387291,
 0.0009308087755925953,
 -0.03405208885669708,
 

In [14]:
# Review vector lengths
len(vector1), len(vector2), len(vector3)

(1536, 1536, 1536)

In [15]:
# Apply dot product to each vector to quantify similarity
np.dot(vector1, vector2), np.dot(vector1, vector3), np.dot(vector2, vector3)

(0.8791284497943928, 0.80002358287471, 0.7934993700101873)

In [16]:
# Review vectors created have a magnitude of 1 with linear algebra function
np.linalg.norm(vector1), np.linalg.norm(vector2), np.linalg.norm(vector3)

(0.9999999518969219, 0.9999999432048748, 0.9999999688261215)